# Google Colaboratory の準備

本ファイルは、`notebook/1-2_create_dataset_metadata.ipynb`をGoogle Colaboratoryで実行するためのノートブックです。  
あらかじめ、`notebook/colab/1-0_prepare_colab.ipynb`の手順を実行し、Googleドライブにリファレンス環境（ソフトウェア）のリポジトリとデータセットの準備が完了していることを確認してください。

## GPUの指定

Google Colaboratoryのメニューバーの「ランタイム」から「ランタイムのタイプを変更」を選択し、
ハードウェア アクセラレータを**GPU**に指定した上で、本ファイルを実行してください。

## ライブラリのインストール

In [ ]:
!pip install -q torch==1.7.1+cu110 torchvision==0.8.2+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q nuscenes-devkit

!pip install -q fire tensorboardX protobuf opencv-python
!pip install -q numba scikit-image scipy pillow

     |███████████████████████         | 834.1 MB 1.1 MB/s eta 0:04:42tcmalloc: large alloc 1147494400 bytes == 0x38f68000 @  0x7fa50bead615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 69.2 MB/s eta 0:00:02tcmalloc: large alloc 1434370048 bytes == 0x7d5be000 @  0x7fa50bead615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 1156.7 MB 1.2 MB/s eta 0:00:01tcmalloc: large alloc 1445945344 bytes == 0xd2daa000 @  0x7fa50bead615 0x592b76 0x4df71e 0x59afff 0x515655 0

In [ ]:
# spconv v1.2.1 のインストール

!git clone --depth 1 --recursive  -b v1.2.1 https://github.com/traveller59/spconv.git
%cd spconv
!SPCONV_FORCE_BUILD_CUDA=1 python setup.py bdist_wheel
!pip install ./dist/spconv*

Cloning into 'spconv'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 118 (delta 16), reused 33 (delta 1), pack-reused 0
Receiving objects: 100% (118/118), 234.67 KiB | 1.71 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Submodule 'third_party/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'third_party/cutlass'
Submodule 'third_party/mp11' (https://github.com/boostorg/mp11) registered for path 'third_party/mp11'
Submodule 'third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'third_party/pybind11'
Cloning into '/content/spconv/third_party/cutlass'...
remote: Enumerating objects: 16512, done.        
remote: Counting objects: 100% (567/567), done.        
remote: Compressing objects: 100% (297/297), done.        
remote: Total 16512 (delta 257), reused 529 (delta 249), pack-reused 15945        
Receiving objects: 100% (16512/1651

## Googleドライブをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# WORK_DIRのパスは自身の作業ディレクトリに変更してください。

WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/work/"
%cd $WORK_DIR

# パスを合わせるため、リポジトリのnotebookディレクトリに移動
%cd ./6th-ai-reference2/notebook

/content/drive/MyDrive/Colab Notebooks/work
/content/drive/MyDrive/Colab Notebooks/work/6th-ai-reference2/notebook


以下より、`notebook/1-2_create_dataset_metadata.ipynb`と同じ内容です。

# 概要

本ファイルは、データセット(3d_labels)で3D物体検出モデルの学習/推論を実行するための事前準備を行います。

> 本ファイルで実行するスクリプトはsecond.pytorchをベースとしています。

## Output

本ファイルを実行することで、`data/3d_labels_painted`ディレクトリに、学習と推論に必要なデータセットのメタ情報(pkl)を生成します。

```
└ data
    ├ 3d_labels
    └ 3d_labels_painted
        ├ maps
        ├ samples
            ├ CAM_FRONT
            └ LIDAR_TOP
        ├ v1.0-trainval
        ├ gt_database        <- 本ファイルにて生成されます
        ├ infos_train.pkl    <- 本ファイルにて生成されます
        ├ infos_val.pkl      <- 本ファイルにて生成されます
        └ kitti_dbinfos_train.pkl  <- 本ファイルにて生成されます
```

# 準備

あらかじめ、`data`ディレクトリに、`3d_labels_painted`ディレクトリがあることを確認してください。

```
└ data
    ├ 3d_labels
    └ 3d_labels_painted      <- 1-1_painting_colab_colab.ipynb にて生成
        ├ maps
        ├ samples
            ├ CAM_FRONT
            └ LIDAR_TOP
        └ v1.0-trainval
```

# データセットの前処理

In [ ]:
dataset_path = "../data/3d_labels_painted"

!python ../second.pytorch/second/create_data.py nuscenes_data_prep \
        --root_path=$dataset_path \
        --version="v1.0-trainval" \
        --dataset_name="NuScenesDataset" \
        --max_sweeps=0 \
        --num_point_features=7

Loading NuScenes tables for version v1.0-trainval...
20 category,
8 attribute,
4 visibility,
2176 instance,
2 sensor,
74 calibrated_sensor,
2475 ego_pose,
37 log,
37 scene,
2475 sample,
4950 sample_data,
42142 sample_annotation,
37 map,
Done loading in 0.661 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
total scene num: 37
exist scene num: 37
train scene: 35, val scene: 2
[100.0%][===================>][429.58it/s][00:32>00:00]   
train sample: 2288, val sample: 187
[100.0%][===================>][0.65it/s][54:09>00:01]   
load 866 bicycle database infos
load 12439 car database infos
load 117 barrier database infos
load 24 construction_vehicle database infos
load 2431 pedestrian database infos
load 1012 traffic_cone database infos
load 2380 truck database infos
load 178 motorcycle database infos
load 4 animal database infos
load 61 static_object.bicycle_rack database infos
load 152 bus database infos
load 5126 static.manmade database infos
load 25 movable_object.deb

以上